In [42]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE
import sklearn.metrics as mt
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from imblearn.combine import SMOTEENN
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score
import math

# record start time
start = time.time()

le = LabelEncoder()
# Importing the dataset
dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/telco-non-internet.csv')
dataset=dataset.dropna()
#dataset=dataset.fillna(0)

dataset['state']=le.fit_transform(dataset['state'])
dataset['area_code']=le.fit_transform(dataset['area_code'])
dataset['international_plan']=le.fit_transform(dataset['international_plan'])
dataset['voice_mail_plan']=le.fit_transform(dataset['voice_mail_plan'])
dataset['churn']=le.fit_transform(dataset['churn'])

#filling missing value with mean
#dataset['TotalCharges'].replace(to_replace = 0, value = dataset['TotalCharges'].mean(), inplace=True)
X=dataset.drop(['churn'],axis=1)
y=dataset["churn"]
#imblearn SMOTE
#smote_enn = SMOTE()
smote_enn = SMOTEENN()
X, y = smote_enn.fit_resample(X, y)

#smote_enn = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
#X, y = smote_enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)


#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)

end = time.time()

print("The time of execution of preprocess:",
      (end-start), "s")



The time of execution of preprocess: 1.157918930053711 s


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-telco2-dt-cs-auc.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-telco2-dt-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco2"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.876293 using {'class_weight': {0: 90, 1: 100}}
0.871114 (0.011928) with: {'class_weight': {0: 10, 1: 100}}
0.873577 (0.011606) with: {'class_weight': {0: 20, 1: 100}}
0.871111 (0.010601) with: {'class_weight': {0: 30, 1: 100}}
0.873517 (0.007720) with: {'class_weight': {0: 40, 1: 100}}
0.871282 (0.012194) with: {'class_weight': {0: 50, 1: 100}}
0.875778 (0.011968) with: {'class_weight': {0: 60, 1: 100}}
0.874753 (0.010596) with: {'class_weight': {0: 70, 1: 100}}
0.875514 (0.009664) with: {'class_weight': {0: 80, 1: 100}}
0.876293 (0.012026) with: {'class_weight': {0: 90, 1: 100}}
0.875152 (0.009106) with: {'class_weight': {0: 100, 1: 100}}


In [53]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-telco2-dt-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-telco2-dt-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco2"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.883554 using {'class_weight': {0: 100, 1: 100}}
0.871060 (0.011960) with: {'class_weight': {0: 10, 1: 100}}
0.874732 (0.012167) with: {'class_weight': {0: 20, 1: 100}}
0.873634 (0.010166) with: {'class_weight': {0: 30, 1: 100}}
0.876314 (0.009383) with: {'class_weight': {0: 40, 1: 100}}
0.875915 (0.014611) with: {'class_weight': {0: 50, 1: 100}}
0.877164 (0.008142) with: {'class_weight': {0: 60, 1: 100}}
0.878925 (0.011387) with: {'class_weight': {0: 70, 1: 100}}
0.880524 (0.012364) with: {'class_weight': {0: 80, 1: 100}}
0.882432 (0.013259) with: {'class_weight': {0: 90, 1: 100}}
0.883554 (0.010761) with: {'class_weight': {0: 100, 1: 100}}


In [54]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-telco2-dt-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-telco2-dt-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco2"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.876450 using {'class_weight': {0: 90, 1: 100}}
0.869242 (0.010883) with: {'class_weight': {0: 10, 1: 100}}
0.872401 (0.013340) with: {'class_weight': {0: 20, 1: 100}}
0.870456 (0.009702) with: {'class_weight': {0: 30, 1: 100}}
0.872210 (0.007601) with: {'class_weight': {0: 40, 1: 100}}
0.871258 (0.013053) with: {'class_weight': {0: 50, 1: 100}}
0.873627 (0.013995) with: {'class_weight': {0: 60, 1: 100}}
0.874219 (0.011063) with: {'class_weight': {0: 70, 1: 100}}
0.873933 (0.011441) with: {'class_weight': {0: 80, 1: 100}}
0.876450 (0.012995) with: {'class_weight': {0: 90, 1: 100}}
0.875930 (0.007711) with: {'class_weight': {0: 100, 1: 100}}


In [55]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-telco2-lr-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-telco2-lr-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco2"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.768213 using {'class_weight': {0: 10, 1: 100}}
0.768213 (0.015582) with: {'class_weight': {0: 10, 1: 100}}
0.766980 (0.014882) with: {'class_weight': {0: 20, 1: 100}}
0.766466 (0.015718) with: {'class_weight': {0: 30, 1: 100}}
0.765131 (0.014656) with: {'class_weight': {0: 40, 1: 100}}
0.765392 (0.015092) with: {'class_weight': {0: 50, 1: 100}}
0.765423 (0.015689) with: {'class_weight': {0: 60, 1: 100}}
0.763444 (0.014694) with: {'class_weight': {0: 70, 1: 100}}
0.762251 (0.013967) with: {'class_weight': {0: 80, 1: 100}}
0.762119 (0.014242) with: {'class_weight': {0: 90, 1: 100}}
0.761547 (0.013674) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [56]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-telco2-lr-cs-f1.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-telco2-lr-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco2"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.674082 using {'class_weight': {0: 100, 1: 100}}
0.405552 (0.006510) with: {'class_weight': {0: 10, 1: 100}}
0.452107 (0.019034) with: {'class_weight': {0: 20, 1: 100}}
0.501629 (0.013411) with: {'class_weight': {0: 30, 1: 100}}
0.539654 (0.012375) with: {'class_weight': {0: 40, 1: 100}}
0.583379 (0.014990) with: {'class_weight': {0: 50, 1: 100}}
0.610912 (0.013192) with: {'class_weight': {0: 60, 1: 100}}
0.631869 (0.013927) with: {'class_weight': {0: 70, 1: 100}}
0.649816 (0.014531) with: {'class_weight': {0: 80, 1: 100}}
0.665596 (0.012340) with: {'class_weight': {0: 90, 1: 100}}
0.674082 (0.012292) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [57]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-telco2-lr-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-telco2-lr-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco2"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.641171 using {'class_weight': {0: 100, 1: 100}}
0.140064 (0.022505) with: {'class_weight': {0: 10, 1: 100}}
0.257991 (0.038506) with: {'class_weight': {0: 20, 1: 100}}
0.350735 (0.022456) with: {'class_weight': {0: 30, 1: 100}}
0.414552 (0.018195) with: {'class_weight': {0: 40, 1: 100}}
0.483848 (0.022308) with: {'class_weight': {0: 50, 1: 100}}
0.528321 (0.017892) with: {'class_weight': {0: 60, 1: 100}}
0.563873 (0.018649) with: {'class_weight': {0: 70, 1: 100}}
0.594797 (0.017933) with: {'class_weight': {0: 80, 1: 100}}
0.622895 (0.014932) with: {'class_weight': {0: 90, 1: 100}}
0.641171 (0.013446) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [58]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-telco2-svm-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-telco2-svm-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco2"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.918689 using {'class_weight': {0: 100, 1: 100}}
0.890021 (0.014775) with: {'class_weight': {0: 10, 1: 100}}
0.894717 (0.014208) with: {'class_weight': {0: 20, 1: 100}}
0.901152 (0.012164) with: {'class_weight': {0: 30, 1: 100}}
0.907859 (0.011341) with: {'class_weight': {0: 40, 1: 100}}
0.911509 (0.010703) with: {'class_weight': {0: 50, 1: 100}}
0.914577 (0.010125) with: {'class_weight': {0: 60, 1: 100}}
0.916266 (0.009744) with: {'class_weight': {0: 70, 1: 100}}
0.917277 (0.009482) with: {'class_weight': {0: 80, 1: 100}}
0.918079 (0.009468) with: {'class_weight': {0: 90, 1: 100}}
0.918689 (0.009422) with: {'class_weight': {0: 100, 1: 100}}


In [59]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-telco2-svm-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-telco2-svm-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco2"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.829476 using {'class_weight': {0: 100, 1: 100}}
0.385383 (0.001606) with: {'class_weight': {0: 10, 1: 100}}
0.458750 (0.008687) with: {'class_weight': {0: 20, 1: 100}}
0.610998 (0.011509) with: {'class_weight': {0: 30, 1: 100}}
0.704444 (0.018592) with: {'class_weight': {0: 40, 1: 100}}
0.753049 (0.013013) with: {'class_weight': {0: 50, 1: 100}}
0.782009 (0.014713) with: {'class_weight': {0: 60, 1: 100}}
0.799315 (0.011246) with: {'class_weight': {0: 70, 1: 100}}
0.814671 (0.012079) with: {'class_weight': {0: 80, 1: 100}}
0.823944 (0.013707) with: {'class_weight': {0: 90, 1: 100}}
0.829476 (0.014155) with: {'class_weight': {0: 100, 1: 100}}


In [60]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-telco2-svm-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-telco2-svm-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco2"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.829533 using {'class_weight': {0: 100, 1: 100}}
0.020778 (0.025885) with: {'class_weight': {0: 10, 1: 100}}
0.269691 (0.016771) with: {'class_weight': {0: 20, 1: 100}}
0.506694 (0.015044) with: {'class_weight': {0: 30, 1: 100}}
0.635235 (0.022519) with: {'class_weight': {0: 40, 1: 100}}
0.705073 (0.015145) with: {'class_weight': {0: 50, 1: 100}}
0.750040 (0.016396) with: {'class_weight': {0: 60, 1: 100}}
0.778738 (0.012236) with: {'class_weight': {0: 70, 1: 100}}
0.803119 (0.013309) with: {'class_weight': {0: 80, 1: 100}}
0.819035 (0.014570) with: {'class_weight': {0: 90, 1: 100}}
0.829533 (0.014594) with: {'class_weight': {0: 100, 1: 100}}
